In [30]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split,Dataset,Subset
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import optuna
import joblib

讀取檔案

In [32]:
train=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/train.csv")
test=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/test.csv")

建立Dataset Class

In [33]:
class FADataset(Dataset):
    def __init__(self, data,train=True):
        super().__init__()
        if(train):
            self.x=data.drop(['id','y'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            self.y=torch.tensor(data['y'].values,dtype=torch.float32)
        else:
            self.x=data.drop(['id'],axis=1).values
            self.x=torch.tensor(self.x,dtype=torch.float32)
            
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        data=self.x[idx]
        target=self.y[idx]
        return data,target,idx


In [131]:
dataset=FADataset(train,train=True)
trainset, validset = random_split(dataset, [0.9, 0.1])
testset=FADataset(test,train=False)
test_dl=DataLoader(testset,shuffle=False, batch_size=80)

建立MLP Class

In [132]:
class MLP(nn.Module):
    def __init__(self,params):
        super().__init__()
        self.model=nn.Sequential(nn.Linear(2,params['num_hidden1']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden1'],params['num_hidden2']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden2'],params['num_hidden3']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden3'],params['num_hidden4']),
                                nn.ReLU(True),
                                nn.Linear(params['num_hidden4'],params['num_hidden5']),
                                nn.ReLU(True),  
                                nn.Linear(params['num_hidden5'],params['num_hidden6']),
                                nn.Linear(params['num_hidden6'],1)
                                )
    def forward(self, x):
        return self.model(x)


Train and evaluate model

In [103]:
def train_and_evaluate(params,model,datadl):

    optimizer = getattr(torch.optim, params['optimizer'])(model.parameters(), lr= params['learning_rate'])
    criterion = nn.MSELoss()

    losses=[]
    num_epochs=50
    model.train()
    for epoch in range(num_epochs):
        for batch,(data, target,idx) in enumerate(datadl):
            outputs = model(data)
            loss = criterion(outputs.reshape(-1), target.reshape(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


    model.eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(datadl):
        outputs = model(data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    # print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)
    
    return model,loss


使用optuna選取較佳的參數

In [104]:

def objective(trial):
        
        params = {

            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
            'optimizer': trial.suggest_categorical("optimizer", ["Adam", "SGD"]),
            'batch_size': trial.suggest_int("batch_size", 64, 320,step=16),
            'num_hidden1': trial.suggest_int("num_hidden1", 50, 500),
            'num_hidden2': trial.suggest_int("num_hidden2", 50, 600),
            'num_hidden3': trial.suggest_int("num_hidden3", 50, 600),
            'num_hidden4': trial.suggest_int("num_hidden4", 50, 600),
            'num_hidden5': trial.suggest_int("num_hidden5", 50, 600),
            'num_hidden6': trial.suggest_int("num_hidden6", 50, 600)
            }
        
        train_dl= DataLoader(trainset, shuffle=True, batch_size=params['batch_size'])
        valid_dl= DataLoader(validset, shuffle=True, batch_size=params['batch_size'])

        model= MLP(params)
        best_model,loss=train_and_evaluate(params,model,train_dl)
      
        torch.save(best_model.state_dict(), f"model_trial_{trial.number}.pth")
    
        return loss


利用optuna選出來的參數建立模型

In [105]:
train_dl= DataLoader(trainset, shuffle=True, batch_size=study_alldata.best_params['batch_size'])
valid_dl= DataLoader(validset, shuffle=True, batch_size=study_alldata.best_params['batch_size'])
criterion = nn.MSELoss()

In [133]:
model=[]
for i in range(8):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study.optimize(objective, n_trials=30)
    print(f"model_{i} loss=",study.best_value)
    model.append(MLP(study.best_params))
    model[i].load_state_dict(torch.load(f"C:/Users/MJ/Desktop/FunctionApproximation/model_trial_{study.best_trial.number}.pth"))
    torch.save(model[i],f'model_{i}.pth')

model_0 loss= 0.022552061745006104
model_1 loss= 0.02433023564517498
model_2 loss= 0.023560093057652314
model_3 loss= 0.02435948667426904
model_4 loss= 0.02350031938403845
model_5 loss= 0.023195896688848736
model_6 loss= 0.02258224644493766
model_7 loss= 0.02255336705433286


Load model

In [ ]:
# model=[]
# for i in range(5):
#     model.append(torch.load(f"C:/Users/MJ/Desktop/FunctionApproximation/model_{i}.pth"))

8個模型分別對於validation dataset 的表現

In [135]:
for i in range(8):
    model[i].eval()
    losses=[]
    for batch,(data, target,idx) in enumerate(valid_dl):
        outputs = model[i](data)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)

loss: 0.020341 
loss: 0.021502 
loss: 0.021000 
loss: 0.022430 
loss: 0.020994 
loss: 0.021126 
loss: 0.019786 
loss: 0.020275 


使用optuna尋找8個模型較佳的權重值

In [149]:
def objective_en(trial):
  
    a=trial.suggest_float('a',0, 100)
    b=trial.suggest_float('b',0, 100)
    c=trial.suggest_float('c',0, 100)
    d=trial.suggest_float('d',0, 100)
    e=trial.suggest_float('e',0, 100)
    f=trial.suggest_float('f',0, 100)
    g=trial.suggest_float('g',0, 100)
    h=trial.suggest_float('h',0, 100)
    
    losses=[]
    
    for batch,(data, target,idx) in enumerate(train_dl):
        outputs = (a*model[0](data)+b*model[1](data)+c*model[2](data)+d*model[3](data)+e*model[4](data)+e*model[4](data)+f*model[5](data)+g*model[6](data)+h*model[7](data))/(a+b+c+d+e+g+h)
        loss = criterion(outputs.reshape(-1), target.reshape(-1))
        losses.append(loss.item())
    # print(f"loss: {sum(losses)/len(losses):>7f} ")
    loss=sum(losses)/len(losses)

    return loss

study_en = optuna.create_study(direction='minimize')
study_en.optimize(objective_en, n_trials=100)


In [153]:
study_en.best_value

0.0221860954884143

使用optuna選出來的權重預測 test data

In [156]:
a=study_en.best_params['a']
b=study_en.best_params['b']
c=study_en.best_params['c']
d=study_en.best_params['d']
e=study_en.best_params['e']
e=study_en.best_params['e']
f=study_en.best_params['f']
g=study_en.best_params['g']
h=study_en.best_params['h']


for i in range(len(testset)):
    output.append(((a*model[0](testset.x[i])+b*model[1](testset.x[i])+c*model[2](testset.x[i])+d*model[3](testset.x[i])+e*model[4](testset.x[i])+f*model[5](testset.x[i])+g*model[6](testset.x[i])+h*model[7](testset.x[i]))/(a+b+c+d+e+g+h)).item())



將test data預測結果存取成 sample_submission.csv

In [157]:
sample_submission=pd.read_csv("C:/Users/MJ/Desktop/FunctionApproximation/sample_submission.csv")
sample_submission['y']=output
sample_submission.to_csv("C:/Users/MJ/Desktop/sample_submission.csv",index=False)

將loss值較大的當成noise data後去除再訓練模型(但此方法loss值大於使用ensemble learning，因此不採用)

In [ ]:
# model=[]
# optimizer=[]

# for i in range(3):
#     model.append(MLP(input_dim=2, num_hidden1=380,num_hidden2=155,num_hidden3=22,num_hidden4=467,num_hidden5=27,num_hidden6=130, output_dim=1))
#     optimizer.append(torch.optim.SGD(model[i].parameters(), lr=0.08115999290485583))
# criterion = nn.MSELoss()

In [ ]:
# model[0].eval()
# indexF=[]
# indexT=[]
# for i in range(len(trainset)):
#     outputs = model[0](trainset.dataset.x[i])
#     loss = criterion(outputs,trainset.dataset.y[i].reshape(1))
#     if(loss<0.08):           #將loss值大於0.08的去除
#         indexT.append(i)

In [ ]:
# Tdataset=torch.utils.data.Subset(trainset, indexT)
# Tdl=DataLoader(Tdataset, shuffle=True, batch_size=batch_size)


# num_epochs = 80
# size = len(Tdl)
# model[1].train()
# for epoch in range(num_epochs):
#     for batch,(data, target,idx) in enumerate(Tdl):
#         outputs = model[1](data)
#         loss = criterion(outputs.reshape(-1), target.reshape(-1))
#         optimizer[1].zero_grad()
#         loss.backward()
#         optimizer[1].step()
#         if batch % 80 == 0: 
#             print(f"loss: {loss.item():>7f} ")
#             losses.append(loss.item())